<a href="https://colab.research.google.com/github/Yesserch-web/habit-tracker/blob/main/MINI_PROJECT01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Vérifiez les fichiers importés dans le répertoire de travail
print("Contenu du répertoire actuel :")
print(os.listdir('/content'))


Contenu du répertoire actuel :
['.config', 'Covid_dataset', 'Covid_dataset.zip', '.ipynb_checkpoints', 'Evaluation Set.zip', 'sample_data']


In [ ]:
import zipfile

zip_path = '/content/Covid_dataset.zip'  # Remplacez par le nom de votre fichier
extract_to = '/content/Covid_dataset'   # Répertoire cible

# Décompression
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Le fichier a été décompressé dans : {extract_to}")
print("Contenu :", os.listdir(extract_to))


BadZipFile: File is not a zip file

In [ ]:
import mimetypes

# Vérifiez le type MIME du fichier
mime_type, _ = mimetypes.guess_type(zip_path)
print(f"Le type MIME du fichier est : {mime_type}")


Le type MIME du fichier est : application/zip


In [ ]:
import zipfile

zip_path = '/content/Covid_dataset (3).zip'  # Remplacez par le nom de votre fichier
extract_to = '/content/Covid_dataset (3)'     # Répertoire cible

# Décompression
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Le fichier a été décompressé dans : {extract_to}")
print("Contenu :", os.listdir(extract_to))


Le fichier a été décompressé dans : /content/Covid_dataset (3)
Contenu : ['train', '__MACOSX']


In [ ]:
import os

train_dir = '/content/Covid_dataset (3)/train'
print("Contenu de 'train' :", os.listdir(train_dir))


Contenu de 'train' : ['CovidNegative', 'CovidPositive', 'ZKEZEFXA', '.DS_Store']


In [ ]:
import zipfile
import os

# Chemin vers le fichier zip
zip_path = '/content/Evaluation Set (1).zip'
extract_to = '/content/Evaluation_Set'  # Dossier où vous souhaitez extraire le contenu

# Décompression
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Vérification du contenu
print("Contenu du dossier extrait : ", os.listdir(extract_to))


Contenu du dossier extrait :  ['__MACOSX', 'Evaluation Set']


In [8]:
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image

# Décompression du fichier d'évaluation
zip_path = '/content/Evaluation Set (1).zip'
extract_to = '/content/Evaluation_Set'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Vérification du contenu
print("Contenu du dossier extrait : ", os.listdir(extract_to))

# Configuration
train_dir = '//content/sample_data/Covid_dataset (3).zip'  # Nouveau chemin pour les données d'entraînement
eval_dir ='/content/sample_data/Evaluation Set (1).zip'  # Nouveau chemin pour le dossier d'évaluation
output_file = "result.txt"
batch_size = 16
num_epochs = 10
learning_rate = 0.001
image_size = 224

# Dataset Class
class ChestXrayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.labels = {"CovidPositive": 1, "CovidNegative": 0}

        # Vérification du contenu des sous-dossiers
        print(f"Vérification des sous-dossiers dans {root_dir}:")
        for label in self.labels:
            folder_path = os.path.join(root_dir, label)
            if os.path.isdir(folder_path):
                print(f"Le dossier {label} existe.")
                for img_name in os.listdir(folder_path):
                    img_path = os.path.join(folder_path, img_name)
                    self.samples.append((img_path, self.labels[label]))
            else:
                print(f"Le dossier {label} n'existe pas !")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Transforms
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets et DataLoaders
train_dataset = ChestXrayDataset(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Modèle (ResNet18 pré-entraîné)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Classification binaire

# Loss et Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Boucle d'entraînement
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Évaluation
model.eval()
eval_images = sorted(os.listdir(eval_dir))
results = []

with torch.no_grad():
    for img_name in eval_images:
        img_path = os.path.join(eval_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)

        output = model(image)
        _, predicted = torch.max(output, 1)
        results.append((img_name, predicted.item()))

# Sauvegarder les résultats
with open(output_file, "w") as f:
    for img_name, label in results:
        f.write(f"{img_name} {label}\n")

print(f"Results saved to {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/Evaluation Set (1).zip'

In [9]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image


def extract_zip(zip_path, extract_to):
    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"Le fichier ZIP '{zip_path}' n'existe pas. Vérifiez le chemin.")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"✅ Extraction terminée : {extract_to}")


train_zip_path = '/content/sample_data/Covid_dataset (3).zip'
eval_zip_path = '/content/sample_data/Evaluation Set (1).zip'


train_extract_to = '/content/Covid_dataset'
eval_extract_to = '/content/Evaluation_Set'


extract_zip(train_zip_path, train_extract_to)
extract_zip(eval_zip_path, eval_extract_to)


train_dir = '/content/Covid_dataset/train'
eval_dir = '/content/Evaluation_Set/Evaluation Set'
output_file = "result.txt"
batch_size = 16
num_epochs = 10
learning_rate = 0.001
image_size = 224


class ChestXrayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.labels = {"CovidPositive": 1, "CovidNegative": 0}

        for label in self.labels:
            folder_path = os.path.join(root_dir, label)
            if not os.path.exists(folder_path):
                raise FileNotFoundError(f"Le dossier '{folder_path}' n'existe pas !")
            for img_name in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_name)
                self.samples.append((img_path, self.labels[label]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = ChestXrayDataset(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


model.eval()
eval_images = sorted(os.listdir(eval_dir))
results = []

with torch.no_grad():
    for img_name in eval_images:
        img_path = os.path.join(eval_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)

        output = model(image)
        _, predicted = torch.max(output, 1)
        results.append((img_name, predicted.item()))


with open(output_file, "w") as f:
    for img_name, label in results:
        f.write(f"{img_name} {label}\n")

print(f"✅ Résultats enregistrés dans le fichier : {output_file}")


✅ Extraction terminée : /content/Covid_dataset
✅ Extraction terminée : /content/Evaluation_Set


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 80.8MB/s]


Epoch [1/10], Loss: 0.2542
Epoch [2/10], Loss: 0.1246
Epoch [3/10], Loss: 0.0253
Epoch [4/10], Loss: 0.0208
Epoch [5/10], Loss: 0.0458
Epoch [6/10], Loss: 0.0104
Epoch [7/10], Loss: 0.0026
Epoch [8/10], Loss: 0.0028
Epoch [9/10], Loss: 0.0006
Epoch [10/10], Loss: 0.0012
✅ Résultats enregistrés dans le fichier : result.txt


In [5]:
import os
import zipfile

# Define the file paths
zip_path = '/content/Evaluation Set (1).zip'
extract_to = '/content/Evaluation_Set'

# Check if the ZIP file exists
if not os.path.exists(zip_path):
    print(f"❌ Le fichier ZIP '{zip_path}' n'existe pas dans l'environnement Colab.")
    print("💡 Assurez-vous d'avoir importé le fichier correctement.")
    print("💡 Utilisez le bouton '📤 Importer' dans Colab ou vérifiez le chemin.")
    print("📂 Contenu actuel de '/content':", os.listdir('/content'))
else:
    print(f"✅ Le fichier ZIP '{zip_path}' a été trouvé.")
    print("🔄 Décompression en cours...")

    # Extract the ZIP file
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"✅ Décompression terminée. Les fichiers sont extraits dans '{extract_to}'.")
        print("📂 Contenu de l'extraction :", os.listdir(extract_to))
    except zipfile.BadZipFile:
        print("❌ Erreur : Le fichier n'est pas un fichier ZIP valide.")



❌ Le fichier ZIP '/content/Evaluation Set (1).zip' n'existe pas dans l'environnement Colab.
💡 Assurez-vous d'avoir importé le fichier correctement.
💡 Utilisez le bouton '📤 Importer' dans Colab ou vérifiez le chemin.
📂 Contenu actuel de '/content': ['.config', 'sample_data']


In [7]:
import os

# Chemin du dossier d'évaluation
eval_dir = '/content/Evaluation_Set/Evaluation Set'  # Remplacez par le chemin correct

# Compter le nombre de fichiers image
if os.path.exists(eval_dir):
    eval_files = [f for f in os.listdir(eval_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"Nombre total d'images dans le dossier d'évaluation : {len(eval_files)}")
else:
    print(f"Le dossier '{eval_dir}' n'existe pas !")


Le dossier '/content/Evaluation_Set/Evaluation Set' n'existe pas !
